# Chapter 6 학습 관련 기술들

### ■ 학습 내용 
#### <u><b>신경망 학습의 핵심 개념들에 대한 분석</b><br></u>
* <b> 6.1 매개변수 갱신 방법 </b>
<font color="blue"><font size="1"><i>가중치 매개변수의 최적값을 탐색하는 최적화 방법</i></font></font>
 - 확률적 경사 하강법(SSD)
 - 모멘텀
 - AdaGrad
 - Adam
* <b> 6.2 가중치의 초깃값 </b>
<font color="blue"><font size="1"><i>가중치 초깃값을 정하는 방법은 <u>올바른 학습을 하는데 매우 중요함</u></i></font></font>
 - Xavier
 - He
* <b> 6.3 배치 정규화 </b>
<font color="blue"><font size="1"><i>배치 정규화를 이용하면 <u>학습을 빠르게 진행할 수 있으며, 초깃값에 영향을 덜 받게 됨</u></i></font></font>
* <b> 6.4 바른 학습을 위해 </b>
 - 오버피팅
 - 가중치 감소 
  <font color="blue"><font size="1"><i>오버피팅의 대응책인 정규화 방법</i></font></font>
 - 드롭아웃 
 <font color="blue"><font size="1"><i>오버피팅의 대응책인 정규화 방법</i></font></font>
* <b> 6.5 적절한 하이퍼파라미터 값 찾기 </b>
#### <font color="red"><b> => 신경망 (딥러닝) 학습의 효율과 정확도를 높일 수 있음</b></font><br><br>

## 6.1 매개변수 갱신 <font color="blue"><font size="2"><i>가중치 매개변수의 최적값을 탐색하는 최적화 방법</i></font></font>
 - 확률적 경사 하강법(SSD)
 - 모멘텀
 - AdaGrad
 - Adam

* 신경망 학습의 목적 
 - 손실 함수의 값을 가능한 한 낮추는 매개변수를 찾는 것
 - 최적 매개변수를 찾는 문제
 - <b> 최적화<sup>optimization</sup></b>
 
 <br>

* 지금까지 최적의 매개변수 값을 찾는 단서
 - 매개변수의 기울기 (미분)를 이용
 - 매개변수의 기울기를 구해, 기울어진 방향으로 매개변수 값을 갱신하는 일을 몇 번이고 반복해서 점점 최적의 매개변수를 찾음
 - <b> 확률적 경사 하강법(SGD) </b>
 
 <br>

### 6.1.2 확률적 경사 하강법(SGD) 
<img src="1.jpg">
* w : 갱신할 가중치 매개변수
* ∂L/∂W : W에 대한 손실 함수의 기울기
* n : 학습률
 - 실제로는 0.01이나 0.001과 같은 값을 미리 정해서 사용
* ← : 우변의 값으로 좌변의 값을 갱신함
* <b><font size="3"><font color="Red"> => SGD는 기울어진 방향으로 일정 거리만 가겠다는 단순한 방법 </font></font></b>

In [8]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr;
    
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]

* lr : learning rate (학습률)
 - 인스턴스 변수로 유지
* update(params, grads) 메서드
 - SGD 과정에서 반복해서 불리게 됨
 - params, grads : 딕셔너리 변수
 - params['W1'], grads[W1'] 과 같이 각각 가중치 매개변수와 기울기 저장

In [10]:
network = TwoLayerNet(...)
optimizer = SGD()

for i in range(10000):
    ...
    x_batch, t_batch = get_mini_batch(...) #미니배치
    grads = network.gradient(x_batch, t_batch)
    params = network.params
    optimizer.update(params, grads)
    ...

NameError: name 'TwoLayerNet' is not defined

* optimizer : '최적화를 행하는 자' 라는 뜻의 단어
 - 이 코드에서는 SGD()가 그 역할을 함
 - 매개변수 갱신은 optimizer가 책임지고 수행하기 때문에 
 - <b>optimizer에 매개변수와 기울기 정보만 넘겨주면 됨</b>
 
 <br>
 
* (이처럼) <b>최적화를 담당하는 클래스를 분리해 구현하면 기능을 모듈화하기 좋음</b>
 - ex) 모멘텀이라는 최적화 기법 역시 update(params, grads)라는 공통의 메서드를 갖도록 구현
 - 그때, optimizer = SGD() 문장을 optimizer = Momentum() 으로만 변경하면 모멘텀으로 바꿀 수 있게 됨

### 6.1.3 SGD의 단점
* SGD는 단순하고 구현이 쉽지만, 문제에 따라서 비효율적일 때가 있음

<br>

<img src="2.jpg">

* 이 함수는 '밥그릇'을 x축 방향으로 늘인 듯한 모습 (왼쪽)이고, 실제로 그 등고선은 오른쪽과 같이 x축 방향으로 늘린 타원 (오른쪽)으로 되어 있음

<img src="3.jpg">

* [식 6.2] 함수의 기울기를 그려보면,
 - 기울기는 y축 방향은 크고 x축 방향은 작다는 것이 특징
 - y축 방향은 가파른데 x축 방향은 완만한 것
 
* 여기에서 주의할 점은,
 - [식 6.2]가 최솟값이 되는 장소는 (x,y) = (0,0)이지만
 - [그림 6-2]가 보여주는 기울기 대부분은 (0,0) 방향을 가리키지 않는다는 것

<img src="4.jpg">

* [그림 6-1] 함수에 SGD 적용
 - 탐색 시작하는 장소(초깃값)는 (x,y) = (-7.0, 2.0)으로 시작
 - SGD는 심하게 굽이진 움직임을 보임
 - 상당히 비효율적인 움직임
 - <b> SGD의 단점은 비등방성 함수 (방향에 따라 성질, 즉 여기에서는 기울기가 달라지는 함수)에서는 탐색 경로가 비효율적이라는 것 </b>
 
<img src="5.jpg">

* (다음으로) SGD의 이러한 단점을 개선 및 SGD를 대체하는 세 가지 기법을 소개
 - 모멘텀
 - AdaGrad
 - Adam

### 6.1.4 모멘텀

* <b>모멘텀</b><sup>Momentum</sup>
 - '운동량' 뜻하는 단어
 - 물리 속성과 관련이 있음

<img src="6.jpg">

(SGD와 유사)
* W : 갱신할 가중치 매개변수
* ∂L/∂W : W에 대한 손실 함수의 기울기
* n : 학습률
* <b>v</b> <font color="red"><b>(새로운 변수)</b></font> : 물리에서 말하는 속도<sup>velocity</sup>

<br>

* [식 6.3]
 - 기울기 방향으로 힘을 받아 물체가 가속된다는 물리 법칙을 나타냄
 - ∝v : 물체가 아무런 힘을 받지 않을 때 서서히 하강시키는 역할 (∝: 0.9 등의 값으로 설정), 
   - 물리에서는 지면 마찰이나 공기 저항에 해당
* [그림 6-4]
 - 모멘텀은 공이 그릇의 바닥을 구르는 듯한 움직임을 보임

 
<img src="7.jpg">

In [1]:
class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
        
        def update(self, params, grads):
            if self.v is None:
                self.v = {}
                for key, val in params.items():
                    self.v[key] = np.zeros_like(val)
                    
                    for key in params.keys():
                        self.v[key] = self.momentum*self.v[key] - self.lr*grads[key]
                        params[key] += self.v[key]

* v
 - 인스턴스 변수
 - 물체의 속도
 - 초기화 때는 아무 값도 담지 않고, 
 - update()가 처음 호출될 때 매개변수와 같은 구조의 데이터를 딕셔너리 변수로 저장
* 나머지 부분은 [식 6.3]과 [식 6.4]를 간단히 코드로 옮긴 것

* [그림 6-5] <b>모멘텀의 갱신 경로</b>는 공이 그릇 바닥을 구르듯 움직임
* SGD와 비교하면 '지그재그 정도'가 덜함
 - x축의 힘은 아주 작지만 방향은 변하지 않아서 한 방향으로 일정하게 가속하기 때문
 - y축의 힘은 크지만 위아래로 번갈아 받아서 상충하여 y축 방향의 속도는 안정적이지 않음
 - 전체적으로는 SGD보다 x축 방향으로 빠르게 다가갈 수 있어서 지그재그 움직임을 줄일 수 있음

<img src="8.jpg">

### 6.1.5 AdaGrad

* 신경망 학습에서는 학습률(n) 값이 중요
 - 이 값이 너무 작으면 학습 시간이 너무 길어지고,
 - 반대로 너무 크면 발산하여 올바른 학습을 할 수 없음
* <b>학습률 감소</b><sup>learning rate decay</sup> - 학습률 정하는 효과적인 기술
 - 학습을 진행하면서 학습률을 점자 줄여가는 방법
 - 처음에는 크게 학습하다가 조금씩 작게 학습한다는 얘기 (실제 신경망 학습에 자주 쓰임)
 
<br>

* <b>AdaGrad</b>
 - 학습률을 서서히 낮추는 가장 간단한 방법은, 매개변수 '전체'의 학습률 값을 일괄적으로 낮추는 것
 - 이를 더욱 발전시킨 것 -> Adagrad
 - '각각의' 매개변수에 '맞춤형' 값을 만들어 준다는 개념
 - 개별 매개변수에 적응적으로<sup>adaptive</sup> 학습률을 조정하면서 학습을 진행
 
<br>

<img src="9.jpg">


* AdaGrad의 갱신 방법 (수식적으로 표현)
 - W : 갱신할 가중치 매개변수
 - ∂L/∂W : W에 대한 손실 함수의 기울기
 - n : 학습률
 - <b>h</b> <font color="red"><b>(새로운 변수)</b></font>
   - [식 6.5]에서 보듯 기존 기울기 값을 제곱하여 계속 더해줌 (⊙ 기호는 행렬의 원소별 곱셈을 의미)
   - 매개변수를 갱신할 때 1/루트h 을 곱해 학습률을 조정
   - 매개변수의 원소 중에서 많이 움직인 (크게 갱신된) 원소는 학습률이 낮아진다는 뜻
     - <b>학습률 감소가 매개변수의 원소마다 다르게 적용됨을 뜻함</b>

In [4]:
class AdaGrad:
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None
        
        def update(self, params, grads):
            if self.h is None:
                self.h = {}
                for key, val in params.items():
                    self.h[key] = np.zeros_like(val)
                    
                    for key in params.keys():
                        self.h[key] += grads[key] * grads[key]
                        params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)

* 모멘텀 구현 코드와 거의 유사
* 코드상 바뀐 부분
 - v -> h로 바뀜
 - 마지막 for문에서의 마지막 두줄
   - <font color="red">※ 주의할 점은 <b>1e-7</b> 이라는 작은 값을 더하는 부분 </font>
   - 이 작은 값은 self.h[key]에 0이 담겨 있다 해도 0으로 나누는 사태를 막아줌

* [그림 6-6]을 보면, 최솟값을 향해 효율적으로 움직이는 것을 알 수 있음
 - y축 방향은 기울기가 커서 처음에는 크게 움직이지만, 그 큰 움직임에 비례해 갱신 정도도 큰 폭으로 작아지도록 조정됨
 - 따라서, y축 방향으로 갱신 강도가 약해졌고, 지그재그 움직임이 줄어듦
<img src="10.jpg">

### 6.1.6 Adam

* (Review)
 - 모멘텀 : 공이 그릇을 구르는 듯한 물리 법칙에 따르는 움직임을 보임
 - AdaGrad : 매개변수의 원소마다 적응적으로 갱신 정도를 조정
 
<br>

* <font size"3"><b>Adam</b></font> ('이 두 기법을 융합하면 어떨까'라는 생각을 기초로 만든 기법)
 - 2015년에 제안된 새로운 방법
 - 이론은 다소 복잡하지만, 직관적으로는 모멘텀과 AdaGrad를 융합한 듯한 방법
* Adam의 특징
 - 두 방법의 이점을 조합했기 때문에 매개변수 공간을 효율적으로 탐색 가능
 - 하이퍼파라미터의 '편향 보정'이 진행됨

<br>

* [그림 6-7]과 같이 Adam 갱신 과정도 그릇에서 공이 구르듯 움직임
 - 모멘텀과 비슷한 패턴이지만, 모멘텀 때보다 공의 좌우 흔들림이 적음
  - <b>학습의 갱신 강도를 적응적으로 조정해서 얻는 혜택</b>
<img src="11.jpg">

### 6.1.7 어느 갱신 방법을 이용할 것인가?

* SGD, 모멘텀, AdaGrad, Adam 네 기법의 결과 비교 분석
 - 사용한 기법에 따라 갱신 경로가 다름
 - 그림으로만 보면, AdaGrad가 가장 나은 것 같은데 ...
   1. <font color="blue"><b>결과는 풀어야 할 문제가 무엇이냐에 따라 달라지므로 주의해야 함</b></font>
   2. <font color="blue"><b>(학습률 등의) 하이퍼파라미터를 어떻게 설정하느냐에 따라서도 결과가 바뀜 </b></font>

<br>

* 네 기법 각자의 장단점이 있기 때문에 <u>모든 문제에서 항상 뛰어난 기법이 존재하지 않음</u>
* 본 책에서는,
 - 지금도 많은 연구에서 SGD를 사용
 - 모멘텀과 AdaGrad도 시도해 볼만한 가치가 있는 방법
 - 요즘 많은 분들이 Adam에 만족해하며 사용
   - <font color="red"><b> => 각자의 상황을 고려해 여러 가지로 시도하는 것이 중요 </b></font>

<img src="12.jpg">

### 6.1.8 MNIST 데이터셋으로 본 갱신 방법 비교

* 손글씨 숫자 인식을 대상으로 네 기법 비교 
 - 각 층이 100개의 뉴런으로 구성된 5층 신경망에서 ReLU를 활성화 함수로 사용해 측점
 
<br>

* [그림 6-9]의 결과
 - SGD의 학습 진도가 가장 느림
 - 이 실험에서 주의할 점은, 하이퍼파라미터인 학습률과 신경망의 구조 (층 깊이 등)에 따라 결과가 달라짐
 - 일반적으로 SGD보다 다른 세 기법이 빠르게 학습하고, 때로는 정확도가 높게 나타남

<img src="13.jpg">

## 6.2 가중치의 초기값 <font color="blue"><font size="2"><i>가중치 초깃값을 정하는 방법은 <u>올바른 학습을 하는데 매우 중요함</u></i></font></font>
<font color="blue"><font size="2"><i>신경망 학습에서 특히 중요한 것이 가중치의 초깃값</i></font></font>
<font color="blue"><font size="2"><i>가중치의 초깃값을 무엇으로 설정하느냐가 신경망 학습의 성패를 가름</i></font></font>
 - Xavier
 - He


### 6.2.1초깃값을 0으로 하면?

### 6.2.2 은닉층의 활성화값 분포
<img src="14.jpg">
<img src="15.jpg">
<img src="16.jpg">
<img src="17.jpg">

### 6.2.3 ReLU를 사용할 때의 가중치 초깃값
<img src="18.jpg">

### 6.2.4 MNIST 데이터셋으로 본 가중치 초깃값 비교
<img src="19.jpg">

## 6.3 배치 정규화 <font color="blue"><font size="2"><i>배치 정규화를 이용하면 <u>학습을 빠르게 진행할 수 있으며, 초깃값에 영향을 덜 받게 됨</u></i></font></font>

* (Review)
 - 가중치의 초깃값을 적절히 설정하면 각 층의 활성화값 분포가 적당히 퍼지면서 학습이 원활하게 수행됨

<br>

* <font size="3"><b>배치 정규화</b><sup>Batch Normalization</font>
 - 각 층의 활성화를 적당히 퍼트리도록 '강제로' 하는 아이디어에서 출발한 방법
 - 2015년데 제안된 방법
 - 아직 세상에 나온지 얼마 안된 기법임에도 많은 연구자와 기술자들이 즐겨 사용하는 방법

### 6.3.1 배치 정규화 

* 배치 정규화가 주목받는 이유
 - <b>학습을 빨리 진행할 수 있음</b> (학습 속도 개선)
 - <b>초깃값에 크게 의존하지 않음</b> (골치 아픈 초깃값 선택 장애여 안녕!!)
 - <b>오버피팅을 억제함</b> (드롭아웃 등의 필요성 감소)

<br>

* <font size="3"><b>배치 정규화</b></font>
 - 각 층에서의 활성화값이 적당히 분포되도록 조정하는 것
 - 배치 정규화<sup>Batch Norm</sup> 계층을 신경망에 삽입
 - 학습 시 미니배치를 단위로 정규화
 - 데이터 분포가 평균이 0, 분산이 1이 되도록 정규화
 
<img src="20.jpg">
<img src="21.jpg">

* 미니배치 B = {x1, x2, .... xm}이라는 m개의 입력 데이터의 집합에 대해 평균 μ<sub>B</sub>와 분산 δ<sub>B</sub><sup>2</sup>
* 입력 데이터를 평균이 0, 분산이 1이 되게 (적절한 분포가 되게) 정규화함
* ε 기호<sup>epsilon, 엡실론</sup>는 작은 값(예컨데 10ㄷ-7 등)으로, <u>0으로 나누는 사태를 예방하는 역할</u>
* [식 6.7]
 - 미니배치 입력 데이터를 평균 0, 분산 1인 데이터로 변환하는 역할
 - 이 처리를 활성화 함수의 앞 (혹은 뒤)에 삽입함으로써 <u>데이터 분포가 덜 치우치게 할 수 있음</u>

<img src="22.jpg">

* 배치 정규화 계층마다 이 정규화된 데이터에 고유한 <b>확대<sup>scale</sup>와 이동<sup>shift</sup> 변환</b> 수행
 - γ : 확대 담당
 - β : 이동 담당
 - 두 값은 처음에는 γ = 1, β = 0부터 시작하고, 학습하면서 적합한 값으로 조정해나감
 
<br>

* 배치 정규화의 알고리즘
 - 신경망에서 순전파 때 적용
 - 계산 그래프로는 [그림 6-17] 처럼 그릴 수 있음
 <img src="23.jpg">
 

### 6.3.2 배치 정규화 효과

* MNIST 데이터셋을 사용하여 배치 정규화 계층을 사용할 때와 사용하지 않을 때의 학습 진도가 어떻게 달라지는지 확인 [그림 6-18]
 - 배치 정규화가 학습을 빨리 진전시키고 있음

<img src="24.jpg">

* 초깃값 분포를 다양하게 줘가며 학습 진행이 어떻게 달라지는지 확인 [그림 6-19]
 - 거의 모든 경우에서 <b>배치 정규화를 사용할 때의 학습 진도가 빠른 것으로 나타남</b>
 - (실제로) 배치 정규화를 이용하지 않는 경우엔 <u>초깃값이 잘 분포되어 있지 않으면 학습이 전혀 진행되지 않는 모습 확인</u> 가능

<br>
* Consequently, 
 - <font size="3"><font color="Red"><b> 배치 정규화를 사용하면 학습이 빨라지며, 가중치 초깃값에 크게 의존하지 않아도 됨 </b></font></font>

<img src="25.jpg">



## 6.4 바른 학습을 위해
 - 오버피팅
 - 가중치 감소
<font color="blue"><font size="1"><i>오버피팅의 대응책인 정규화 방법</i></font></font>
 - 드롭아웃 
 <font color="blue"><font size="1"><i>오버피팅의 대응책인 정규화 방법</i></font></font>

## 6.4.1 오버피팅

* <font size="3"><b>오버피팅</b></font>
 - 신경망이 훈련 데이터에만 지나치게 적응되어 그 외의 데이터에는 제대로 대응하지 못하는 상태
 - 기계학습에서 오버피팅의 문제가 되는 일이 많음

<br>

* 오버피팅이 발생할 수 있는 경우
 - 매개변수가 많고 표현력이 높은 모델
 - 훈련 데이터가 적음

<br> 

* 이 두 요건을 일부러 충족하여 오버피팅 발생 예시
 - 60,000개인 MNIST 데이터셋의 훈련 데이터 중 300개만 사용
 - 7층 네트워크를 사용하여 네트워크의 복잡성을 높임
 - 각 층의 뉴런은 100개
 - 활성화 함수는 ReLU를 사용

* 데이터를 읽는 코드

In [12]:
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)
# 오버피팅을 재현하기 위해 학습 데이터 수를 줄입
x_train = x_train[:300]
t_train = t_train[:300]

Done
Done
Done
Done
Converting train-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Creating pickle file ...
Done!


* 훈련을 수행하는 코드
 - 에폭마다 모든 훈련 데이터와 모든 시험 데이터 각각에서 정확도 산출
 - train_acc_list와 test_acc_list에는 에폭 단위(모든 훈련 데이터를 한 번씩 본 단위)의 정확도를 저장

In [20]:
# coding: utf-8
import os
import sys

sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from common.multi_layer_net import MultiLayerNet
from common.optimizer import SGD

network = MultiLayerNet(input_size=784, hidden_size_list=[100, 100, 100, 100, 100, 100], output_size=10)
optimizer = SGD(lr=0.01) # 학습률이 0.01인 SGD로 매개변수 갱신

max_epochs = 201
train_size = x_train.shape[0]
batch_size = 100

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)
epoch_cnt = 0

for i in range(1000000000):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    grads = network.gradient(x_batch, t_batch)
    optimizer.update(network.params, grads)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)

        print("epoch:" + str(epoch_cnt) + ", train acc:" + str(train_acc) + ", test acc:" + str(test_acc))

        epoch_cnt += 1
        if epoch_cnt >= max_epochs:
            break

epoch:0, train acc:0.106666666667, test acc:0.1023
epoch:1, train acc:0.106666666667, test acc:0.1035
epoch:2, train acc:0.103333333333, test acc:0.1067
epoch:3, train acc:0.126666666667, test acc:0.11
epoch:4, train acc:0.146666666667, test acc:0.1188
epoch:5, train acc:0.163333333333, test acc:0.1302
epoch:6, train acc:0.173333333333, test acc:0.1443
epoch:7, train acc:0.216666666667, test acc:0.1639
epoch:8, train acc:0.236666666667, test acc:0.1754
epoch:9, train acc:0.266666666667, test acc:0.1858
epoch:10, train acc:0.283333333333, test acc:0.1981
epoch:11, train acc:0.32, test acc:0.213
epoch:12, train acc:0.336666666667, test acc:0.2305
epoch:13, train acc:0.36, test acc:0.2418
epoch:14, train acc:0.403333333333, test acc:0.2579
epoch:15, train acc:0.413333333333, test acc:0.2675
epoch:16, train acc:0.44, test acc:0.2816
epoch:17, train acc:0.46, test acc:0.2915
epoch:18, train acc:0.516666666667, test acc:0.3151
epoch:19, train acc:0.54, test acc:0.3288
epoch:20, train acc:0.5

* 훈련 데이터를 사용하여 측정한 정확도
 - 100 에폭을 지나는 무렵부터 거의 100%
 - 그러나, 시험 데이터에 대해서는 큰 차이를 보임
 - 이처럼, 정확도가 크게 벌어지는 것은 <font size="3"><font color="red"><b>훈련 데이터에만 적응<sup>fitting</sup>해버린 결과</b></font></font>
 - 훈련 때 사용하지 않은 범용 데이터(시험 데이터)에는 제대로 대응하지 못함
 
<img src="50.JPG">

## 6.4.2 가중치 감소

* <b>가중치 감소</b><sup>weight decay</sup>
 - 학습 과정에서 큰 가중치에 대해서는 그에 상응하는 큰 페널티를 부과하여 오버피팅을 억제하는 방법
   - ※ 오버피팅 : 가중치 매개변수의 값이 커서 발생하는 경우가 많음
* 신경망 학습의 목적 -> 손실 함수의 값을 줄이는 것
* 예시)
 - ex)가중치의 제곱 법칙(L2 법칙)을 손실 함수에 더함 -> <font color="red"><b>가중치가 커지는 것을 억제</b></font>
 - 가중치를 <b>W</b>라 하면 L2 법칙에 따른 가중치 감소는 <b>1/2λW<sup>2</sup></b>이 되고, 이 <b>1/2λW<sup>2</sup></b>을 손실 함수에 더함
 - λ<sup>람다</sup> : 정규화의 세기를 조절하는 하이퍼파라미터
 - λ를 크게 설정할수록 큰 가중치에 대한 페널티가 커짐
 - 1/2λW<sup>2</sup></b> : 1/2은 1/2λW<sup>2</sup></b>의 미분 결과인 λW를 조절하는 역할의 상수
* 가중치 감소는 모든 가중치 각각의 손실 함수에 1/2λW<sup>2</sup></b>을 더함
 - 따라서, <font color="red"><b>가중치의 기울기를 구하는 계산에서는 그동안의 오차역전파법에 따른 결과에 정규화 항을 미분한 λW를 더함</b></font>

<br>

* λ = 0.1로 가중치 감소를 적용한 실험 정확도 추이 분석
 - 훈련 데이터에 대한 정확도와 시험 데이터에 대한 정확도에는 [그림 6-20]과 비교했을 때 여전히 차이는 존재하지만,
 - 가중치 감소를 이용하지 않은 [그림 6-20]과 비교하면 그 차이가 줄어듦을 확인
   - <font color="red"><b>오버피팅이 억제됨</b></font>
 - 더하여, (앞서와 달리) 훈련 데이터에 대한 정확도가 100%(1.0)에 도달하지 못한 점도 주목해야 함

<img src="51.JPG">

## 6.4.3 드롭아웃

* (Overview) 가중치 감소
 - 오버피팅을 억제하는 방식으로 손실 함수에 가중치의 L2 법칙을 더한 방법
 - 간단하게 구현 가능
 - 어느 정도 지나친 학습을 억제할 수 있음
 - 그러나, 신경망 모델이 복잡해지면 가중치 감소만으로는 대응이 어려워짐 
   - 이를, <font size="3"><font color="red"><b>드롭아웃</b></font></font><sup>Dropout</sup> 이라는 기법을 사용하여 해결 가능

<br>

* <font size="3"><b>드롭아웃</b></font><sup>Dropout</sup>
 - 뉴런을 임의로 삭제하면서 학습하는 방법
 - 훈련 때 은닉층의 뉴런을 무작위로 골라 삭제
 - 삭제된 뉴런은 신호를 전달하지 않게됨
 - 훈련 때는 데이터를 흘릴 때마다 삭제할 뉴런을 무작위로 선택하고,
 - 시험 때는 모든 뉴런에 신호를 전달함
 - 단, 시험 때는 각 뉴런의 출력에 훈련 때 삭제한 비율을 곱하여 출력함

<img src="52.JPG">
<img src="53.JPG">

## 6.5 적절한 하이퍼파라미터 값 찾기 <font color="blue"><font size="2"><i>하이퍼파라미터 값 탐색은 <u>최적 값이 존재할 법한 범위를 점차 좁히면서 하는 것이 효과적</u></i></font></font>


<img src="54.JPG">

### ■ Summary 
#### <u><b>신경망 학습의 핵심 개념들에 대한 분석</b><br></u>
* <b> 6.1 매개변수 갱신 방법 </b>
<font color="blue"><font size="1"><i>가중치 매개변수의 최적값을 탐색하는 최적화 방법</i></font></font>
 - 확률적 경사 하강법(SSD)
 - 모멘텀
 - AdaGrad
 - Adam
* <b> 6.2 가중치의 초깃값 </b>
<font color="blue"><font size="1"><i>가중치 초깃값을 정하는 방법은 <u>올바른 학습을 하는데 매우 중요함</u></i></font></font>
 - Xavier
 - He
* <b> 6.3 배치 정규화 </b>
<font color="blue"><font size="1"><i>배치 정규화를 이용하면 <u>학습을 빠르게 진행할 수 있으며, 초깃값에 영향을 덜 받게 됨</u></i></font></font>
* <b> 6.4 바른 학습을 위해 </b>
 - 오버피팅
 - 가중치 감소
  <font color="blue"><font size="1"><i>오버피팅의 대응책인 정규화 방법</i></font></font>
 - 드롭아웃 
 <font color="blue"><font size="1"><i>오버피팅의 대응책인 정규화 방법</i></font></font>
* <b> 6.5 적절한 하이퍼파라미터 값 찾기 </b>
<font color="blue"><font size="1"><i>하이퍼파라미터 값 탐색은 <u>최적 값이 존재할 법한 범위를 점차 좁히면서 하는 것이 효과적</u></i></font></font>
#### <font color="red"><b> => 신경망 (딥러닝) 학습의 효율과 정확도를 높일 수 있음</b></font><br><br>